# How does SuSiE perform given T=200 nonzero effects?

Assume using prior = 0.2. The SuSiE power is very low when we have 200 nonzero effects. 

## Results

In [16]:
dscout.summary

effect_num,pve,mean_corX,power,fdr,cs_size,cs_num,top_hit_rate
200,0.01,0.1347851,0.0000,0.0000,0.00,0.0,0.0000
200,0.02,0.1347851,0.0000,0.0000,0.00,0.0,0.0000
200,0.03,0.1347851,0.0005,0.0000,1.00,0.1,1.0000
200,0.05,0.1347851,0.0010,0.0000,1.00,0.2,1.0000
200,0.10,0.1347851,0.0040,0.0000,1.00,0.8,1.0000
200,0.20,0.1347851,0.0050,0.0909,1.00,1.1,0.9091
200,0.40,0.1347851,0.0085,0.0556,1.05,1.8,0.9444
200,0.50,0.1347851,0.0090,0.1000,1.00,2.0,0.9000
200,0.70,0.1347851,0.0145,0.0645,1.00,3.1,0.9355
200,0.90,0.1347851,0.0210,0.0667,1.00,4.5,0.8889


## Code details

In [2]:
dscout_Q3 = readRDS('dscout_Q3.rds')
dscout_Q3 = dscout_Q3[!is.na(dscout_Q3$sim_gaussian_large.output.file),]
dscout_Q3 = dscout_Q3[!is.na(dscout_Q3$susie_large.output.file),]

In [10]:
dscout_df = data.frame(dscout_Q3$sim_gaussian_large.effect_num, dscout_Q3$sim_gaussian_large.pve, 
                       dscout_Q3$score.hit, dscout_Q3$score.signal_num, dscout_Q3$score.cs_medianSize,
                       dscout_Q3$score.top_hit, dscout_Q3$sim_gaussian_large.mean_corX, dscout_Q3$susie_large.avg_purity)
names(dscout_df) = c('effect_num', 'pve','hit', 'cs_num', 'cs_size', 'top_hit', 'corX', 'avg_purity')

In [12]:
power.summary = aggregate(hit ~ effect_num + pve, dscout_df, sum)
power.summary$power = power.summary$hit / (power.summary$effect_num*10)
fdr.summary = aggregate(cs_num ~ effect_num + pve, dscout_df, sum)
fdr.summary$fdr = round(1 - power.summary$hit / fdr.summary$cs_num, 4)
meannonzero = function(x){mean(x[x!=0])}
setsize.summary = aggregate(cs_size ~ effect_num + pve, dscout_df, meannonzero)
tophit.summary = aggregate(top_hit ~ effect_num + pve, dscout_df, sum)
tophit.summary$tophit_rate = round(tophit.summary$top_hit / fdr.summary$cs_num , 4)
corX.summary = aggregate(corX ~ effect_num + pve, dscout_df, mean)
cs_num.summary = aggregate(cs_num ~ effect_num + pve, dscout_df, mean)

In [13]:
dscout.summary = data.frame(power.summary$effect_num, power.summary$pve, corX.summary$corX,
                            power.summary$power, fdr.summary$fdr, setsize.summary$cs_size, 
                            cs_num.summary$cs_num, tophit.summary$tophit_rate)
names(dscout.summary) = c('effect_num', 'pve', 'mean_corX','power', 
                          'fdr', 'cs_size', 'cs_num','top_hit_rate')

In [15]:
is.nan.data.frame <- function(x)
do.call(cbind, lapply(x, is.nan))
dscout.summary[is.nan(dscout.summary)] = 0